<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/POS/Part_of_Speech_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%82%D0%BA%D0%B0%2C_NER%2C_%D0%B8%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BE%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part-of-Speech разметка, NER, извлечение отношений

## Подключение библиотек

In [ ]:
#@title Ставим все недостоющие библиотеки
!pip install pyconll
!pip install corus
!pip install -U spacy
!python -m spacy download ru_core_news_sm  
!pip install razdel

!pip install -U tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers


In [ ]:
#@title Скачиваем данные для анализа
!mkdir datasets
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip
!rm collection5.zip

In [13]:
import pyconll
import re
import numpy as np
import pandas as pd
import en_core_web_sm
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, NgramTagger
for val in ['punkt', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words']:
  nltk.download(val)

from corus import load_ne5
from razdel import tokenize

from sklearn import model_selection, preprocessing, linear_model
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import spacy
from spacy import displacy
from spacy.lang.ru.examples import sentences 

import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Дополнительные функции

In [3]:
def eval_tagger(train_data, test_data, tagger_cl, backoff=None):
  """
    Функция для оценки UnigramTagger, BigramTagger, TrigramTagger
  """
  tagger = tagger_cl(train_data, backoff=backoff)
  return tagger.evaluate(test_data)

def backoff_tagger(train_data, tagger_cl, backoff=None):
  """
    Функция для создании 3-х этапного таггера
  """
  for cls in tagger_cl:
      backoff = cls(train_data, backoff=backoff)
  return backoff

## Задание 1. Напишем теггер на данных с русским языком

### 1.1 Проверим UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [4]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')
result = {}

In [5]:
train_data = []
test_data = []
test_sent = []

for sent in full_train[:]:
    train_data.append([(token.form, token.upos) for token in sent])

for sent in full_test[:]:
    test_data.append([(token.form, token.upos) for token in sent])
    test_sent.append([(token.form, token.upos) for token in sent])

In [6]:
#@title Для каждого таггера считаем отдельно сначало
for tagger_cl in [UnigramTagger, BigramTagger, TrigramTagger]:
    result[tagger_cl.__name__] = {
        'accuracy': eval_tagger(train_data, test_data, tagger_cl)
    }

In [7]:
#@title Теперь в сочитании таггеры
for tagger_cl in [UnigramTagger, BigramTagger, TrigramTagger]:
    for backoff in [UnigramTagger, BigramTagger, TrigramTagger]:
        if tagger_cl.__name__ == backoff.__name__:
            continue
        back_name = f'{tagger_cl.__name__}/{backoff.__name__}'
        backoff_cl = backoff(train_data)
        result[back_name] = {'accuracy': eval_tagger(
            train_data, test_data, tagger_cl, backoff=backoff_cl)
        }

In [8]:
#@title 3-х этапный таггер - TrigramTagger/BigramTagger/UnigramTagger
tag = backoff_tagger(train_data,
                     [BigramTagger, TrigramTagger],
                     backoff=UnigramTagger(train_data))
result['TrigramTagger/BigramTagger/UnigramTagger'] = {
    'accuracy': tag.evaluate(test_data)
}

### 1.2 Написшем свой теггер, попробуя разные векторайзеры, добавив знание не только букв но и слов

In [9]:
train_tok = []
train_label = []
for sent in train_data[:]:
    for tok in sent:
        train_tok.append(str(tok[0]))
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in test_data[:]:
    for tok in sent:
        test_tok.append(str(tok[0]))
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [10]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [11]:
dict_vect = {
    'HashingVectorizer_char': HashingVectorizer(ngram_range=(1, 5), analyzer='char'),
    'HashingVectorizer_100': HashingVectorizer(n_features=100),
    'CountVectorizer_char': CountVectorizer(ngram_range=(1, 5), analyzer='char'),
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer_char': TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),
    'TfidfVectorizer': TfidfVectorizer()
}

for name_vect, vect in dict_vect.items():
  x_train = vect.fit_transform(train_tok)
  x_test = vect.transform(test_tok)
  model = LogisticRegression(random_state=15)
  model.fit(x_train, train_enc_labels)
  pred = model.predict(x_test)
  result[name_vect] = { 'accuracy': accuracy_score(test_enc_labels, pred) }


In [12]:
result_df = pd.DataFrame(result).transpose().sort_values(by='accuracy', ascending=False)
result_df

,accuracy
CountVectorizer_char,0.942034
TfidfVectorizer_char,0.931766
HashingVectorizer_char,0.924513
BigramTagger/UnigramTagger,0.829279
TrigramTagger/BigramTagger/UnigramTagger,0.829143
TrigramTagger/UnigramTagger,0.828550
UnigramTagger,0.823732
CountVectorizer,0.696159
TfidfVectorizer,0.696152
UnigramTagger/BigramTagger,0.617618


### 1.3 Сравним все реализованные методы и сделаем выводы

**Вывод**

Судя по результатам:
1. Для одиночных теггеров лучше использовать *UnigramTagger* у него точность 0.823732
2. Для множественного теггера лучше использовать *CountVectorizer_char* у него показатель 0.939469

## Задание 2. Проверить насколько хорошо работает NER

### 2.1 Проверим NER из nltk/spacy


In [17]:
txt = "На площади Южного мола 15 ресторанов представили гостям лучшие блюда из своего меню. Более 10 участников-производителей собственной продукции и фермеров предложили сладости, мед, орехи, грибы, напитки, колбасы, сыр, натуральное мороженое. В рамках концертной программы выступили шоу барабанщиков, музыкальные группы, ди-джеи. Летний тур фестиваля будет проходить в течение всего курортного сезона на самых живописных площадках города. За этот период на нем выступят более 90 исполнителей, ди-джеев и музыкальных коллективов. Мероприятия пройдут на площади Южного мола у сочинского Морского порта, на площади курорта «Роза Хутор» в горном кластере Сочи, на набережной ТРЦ «Мандарин» в Адлерском районе. Гостей ждет насыщенная концертно-развлекательная программа и кулинарные мастер-классы. Также в дни фестиваля сочинцы и гости курорта смогут посетить фермерские дворики, интерактивные локации, спортивные и игровые зоны, занятия утренней йогой. Вход на мероприятия свободный."

In [18]:
#@title nltk
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(txt))) if hasattr(chunk, 'label') }

{('Южного', 'PERSON')}

In [19]:
#@title spacy
nlp = spacy.load('ru_core_news_sm')
docs = nlp(txt)
displacy.render(docs, jupyter=True, style='ent')

### 2.2 Напишим свой нер попробовав разные подходы:
  * передаём в сетку токен и его соседей
  * передаём в сетку только токен
  * свой вариант

In [20]:
dir = 'Collection5/'
records = load_ne5(dir)
document = next(records).text
nltk.pos_tag(nltk.word_tokenize(document))[:10]

[('Мособлдума', 'JJ'),
 ('позволила', 'NNP'),
 ('Андрею', 'NNP'),
 ('Воробьеву', 'NNP'),
 ('возглавить', 'NNP'),
 ('правительство', 'NNP'),
 ('Московская', 'NNP'),
 ('областная', 'NNP'),
 ('дума', 'NNP'),
 ('совместила', 'NNP')]

In [21]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [22]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         219060
PER          21184
ORG          13646
LOC           4564
GEOPOLIT      4355
MEDIA         2482
Name: tag, dtype: int64

In [23]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [24]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
train_data = train_data.batch(256)
valid_data = valid_data.batch(256)

In [25]:
train_data = train_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [26]:
vectorize_layer = TextVectorization(
    standardize=None,
    max_tokens=30000,
    output_mode='int',
    output_sequence_length=10)

text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [29]:
class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(30000, 64)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.dr1 = Dropout(0.5)
        self.fc2 = Dense(100, activation='relu')
        self.dr2 = Dropout(0.25)
        self.fc3 = Dense(50, activation='relu')
        self.fc4 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.dr1(fc_x)
        fc_x = self.fc2(fc_x)
        fc_x = self.dr2(fc_x)
        fc_x = self.fc3(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc4(concat_x)
        return prob

In [30]:
mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
778/778 [==============================] - 27s 31ms/step - loss: 0.4013 - accuracy: 0.8812 - val_loss: 0.2323 - val_accuracy: 0.9295
Epoch 2/3
778/778 [==============================] - 24s 31ms/step - loss: 0.1601 - accuracy: 0.9519 - val_loss: 0.2094 - val_accuracy: 0.9398
Epoch 3/3
778/778 [==============================] - 25s 32ms/step - loss: 0.1219 - accuracy: 0.9619 - val_loss: 0.2358 - val_accuracy: 0.8964


In [33]:
vectorize_layer = TextVectorization(
    standardize=None,
    max_tokens=30000,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=10)

text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [34]:
mmodel2 = modelNER()
mmodel2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel2.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
778/778 [==============================] - 27s 35ms/step - loss: 0.3993 - accuracy: 0.8828 - val_loss: 0.2334 - val_accuracy: 0.9294
Epoch 2/3
778/778 [==============================] - 25s 32ms/step - loss: 0.1576 - accuracy: 0.9525 - val_loss: 0.2069 - val_accuracy: 0.9379
Epoch 3/3
778/778 [==============================] - 25s 32ms/step - loss: 0.1222 - accuracy: 0.9617 - val_loss: 0.2071 - val_accuracy: 0.9412


### 3. Сравнить реализованные подходы на качество

**Вывод:**

n-gram показал не самых хороший результат, в отличии от отдельными токенами